# Music recommender 

### Data cleaning and examination

In [2]:
#import necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import re
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [3]:
#load data
df=pd.read_csv('/Users/jeffreyng/Music_Rec_Engine/Music-Recommender-Engine/output-data.csv')

In [256]:
#looking at data
df.artist.value_counts()

Neil Young               27
Bonnie “Prince” Billy    22
Animal Collective        21
Guided by Voices         20
David Bowie              19
                         ..
Shad                      1
Feeding People            1
Illangelo                 1
John Luther Adams         1
Bear Vs. Shark            1
Name: artist, Length: 9049, dtype: int64

In [5]:
#replace nan with misc.
df.genre.replace(np.nan, 'Misc', inplace=True)

In [6]:
#looking at genre breakdown
df.genre.value_counts()

Rock            6879
Electronic      3659
Experimental    1669
Rap             1644
Pop/R&B         1298
Misc            1289
Metal            762
Folk/Country     748
Jazz             282
Global           162
none              11
Name: genre, dtype: int64

### Now we will build our rec engine

In [1]:
#importing surprise package
import surprise

In [257]:
#making our data 
from surprise import Dataset, Reader
reader= Reader()
data = Dataset.load_from_df(df[['artist','album', 'score']], reader)

In [258]:
#TTS
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

In [259]:
#using SVD algo and fitting train data
from surprise import SVD, accuracy
algo = SVD()
algo.fit(trainset)

In [260]:
#making predictions on test_Set
predictions = algo.test(testset)

In [261]:
# getting RMSE 
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 2.3843


2.384302609782183

In [288]:
#looking at predictions data
predictions[:][:][3]

Prediction(uid='DAWN', iid='Blackheart', r_ui=8.0, est=5, details={'was_impossible': False})

In [263]:
predictions[:][:][1]

Prediction(uid='Larkin Grimm', iid='Parplar', r_ui=5.2, est=5, details={'was_impossible': False})

In [264]:
#converting predictions into a df
df1= pd.DataFrame(predictions[:])

In [338]:
df1.loc[2779]

uid                   Beat Happening
iid                      Look Around
r_ui                             8.2
est                                5
details    {'was_impossible': False}
Name: 2779, dtype: object

In [347]:
#function that pulls in an album name and returns top 10 similar albums indices 
def rec_album():
    y=df1.loc[df1['uid'] == 'Beat Happening'] #we enter our album name here
    container=[]
    for ele in df1.r_ui:
        container.append(abs(int(y.r_ui - ele))) #looking at album ratings difference between prediction and actual 
        #finding the most similar
        
    sim= pd.Series(container) #converting list of albums into series
    sorted_sim= sim.sort_values()[0:10] #sorting series and returning top 10
    print(sorted_sim) #printout
    return sorted_sim #top ten sorted


In [349]:
# returning album indices of recommended albums to 'Beat Happening'
rec_album()
# we can then use df1.loc[index] to find our actual album

1246    0
1646    0
3087    0
3685    0
3684    0
1650    0
3682    0
1653    0
1654    0
1655    0
dtype: int64


1246    0
1646    0
3087    0
3685    0
3684    0
1650    0
3682    0
1653    0
1654    0
1655    0
dtype: int64